In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def image_read(image_path):
    img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    if img is None:
        print("Error: Could not read the image.")
        return
    return  img

def show_image(matrix,x ,windowName):
    padded_matrix = np.array(matrix, dtype=np.uint8) *x
    print('padded matrix: ', padded_matrix.shape)
    cv2.imshow(windowName, padded_matrix)
    cv2.waitKey(0)
    cv2.destroyWindow(windowName)


def padded_image(img,padd):
    height, width = img.shape[:2]
    padded_matrix = np.ones((height + 2*padd, width + 2*padd), dtype=np.uint8)*0
    padded_matrix[padd:padd+height, padd:padd+width] = img
    return padded_matrix

def normal_pixel(pix_val, pix_min, pix_max):
    temp= int(255 * (pix_val - pix_min) / (pix_max - pix_min))
    return temp

def frame_image(image,frame_size):
    rows, cols = image.shape
    output_img = np.zeros((rows, cols), dtype=np.uint8)
    padd_size=frame_size//2
    padd_img = padded_image(image, padd_size)
    pix_min = np.min(image)
    pix_max = np.max(image)
    for i in range(padd_size, rows+padd_size):
        #print("Pad image: ",padd_img[i])
        for j in range(padd_size, cols+padd_size):
            frame = padd_img[i - padd_size:i + padd_size + 1, j - padd_size:j + padd_size + 1]
            new_value = np.mean(frame)
            output_img[i - padd_size, j - padd_size] = normal_pixel(new_value,pix_min,pix_max)
        #print("output image: ",output_img[i-padd_size])

    return output_img
def cal_histogram(img):
    rows,cols=img.shape
    arr=np.zeros(256,dtype=int)
    for i in range(rows):
        for j in range(cols):
            arr[img[i][j]]+=1
    return arr
def pdf(img,arr):
    arr1 = np.zeros(len(arr), dtype=float)
    rows, cols = img.shape
    arr1=arr/(rows*cols)
    return arr1
def cummulative_pdf(arr):
    value = 0
    arr1 = arr.copy()
    for i in range(len(arr)):
        value = arr[i] + value
        arr1[i] = value
    return arr1

def transform(cdf):
    cdf=cdf*255
    return cdf
def plot_graph(data, title, xlabel, ylabel, color):
    plt.figure()
    plt.plot(data, color=color)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()


def find_root(eq_list, key):
    if key not in eq_list:
        eq_list[key] = key
        return key
    while eq_list[key] != key:
        eq_list[key] = eq_list[eq_list[key]]  # Path compression
        key = eq_list[key]
    return key
    

def connectivit_8(image,limit,padd_size):
    height, width = image.shape
    padd_img=(image,padd_size)
    connect_8 = np.zeros_like(padd_img, dtype=np.uint8)
    eq_list = {}
    label = 1
    for i in range(padd_size, height):
        for j in range(padd_size, width):
            if image[i, j]>=0 and image[i,j] <=limit:  # Foreground pixel
                neighbors = []
                if connect_8[i, j - padd_size] >0 and connect_8[i, j - padd_size]<=limit:
                    neighbors.append(connect_8[i, j - padd_size])
                if connect_8[i - padd_size, j] > 0 and connect_8[i - padd_size, j]<=limit:
                    neighbors.append(connect_8[i - padd_size, j])
                if connect_8[i - padd_size, j - padd_size] > 0  and connect_8[i - padd_size, j - padd_size]<=limit:
                    neighbors.append(connect_8[i - padd_size, j - padd_size])
                if connect_8[i - padd_size, j + padd_size] > 0  and connect_8[i - padd_size, j + padd_size] <=limit:
                    neighbors.append(connect_8[i - padd_size, j + padd_size])

                if not neighbors:
                    connect_8[i, j] = label
                    eq_list[label] = label
                    label += 1
                else:
                    min_label = min(neighbors)
                    connect_8[i, j] = min_label
                    for neighbor in neighbors:
                        eq_list[find_root(eq_list, neighbor)] = find_root(eq_list, min_label)

    for i in range(height):
        for j in range(width):
            if connect_8[i, j] > 0:
                connect_8[i, j] = find_root(eq_list, connect_8[i, j])

    unique_labels = set(find_root(eq_list, lbl) for lbl in eq_list.values())
    """
    print("Number of labels:", len(unique_labels))
    print("Equivalence list:", eq_list)
    print(f"Unique Vals: {unique_labels}")
    """
    return connect_8, eq_list


def overlap_label(nuc_img,cyto_img):
    rows, cols = nuc_img.shape
    for i in range(rows):
        for j in range(cols):
            if nuc_img[i, j] > 0 and cyto_img[i, j]==0:  # Foreground pixel
                nuc_img[i,j]=0
    return nuc_img
def merge_mask(nuc_img,cyto_img):
    rows, cols = nuc_img.shape
    mask = np.zeros((rows,cols), dtype=np.uint8)
    for i in range(rows):
        for j in range(cols):
            if nuc_img[i, j] == 0 and cyto_img[i, j] >0:  # Foreground pixel
                mask[i, j] = 128
            if nuc_img[i, j] > 0 and cyto_img[i, j] >0:
                mask[i, j] = 255
    return mask
def limit_img(img,x):
    rows,cols=img.shape
    matrix = np.array(img, dtype=np.uint8)
    for i in range(0,rows):
        for j in range (0,cols):
            if img[i][j]>0 :
                matrix[i][j]=x
            else:
                matrix[i][j]=0
    return matrix

def dice_coef(orig_mask,made_mask,coefficent):
    rows, cols = orig_mask.shape
    x = 0
    y = 0
    intersec = 0
    for si in range(rows):
        for j in range(cols):
            if orig_mask[i,j]==coefficent:
                x+=1
            if made_mask[i,j]==coefficent:
                y+=1
            if orig_mask[i,j]==coefficent and made_mask[i,j]==coefficent:
                intersec+=1
    divdend=x+y
    if divdend>0:
        dice_coefficient=(2*intersec)/(x+y)
    else:
        dice_coefficient=0
    return dice_coefficient
for i in range(37,40):
    print("i: ",i)
    temp = str(i).zfill(3)
    image_path=f"E:\\2. DIP\\2. LAB\\pythonProject\\Assignment\\dataset_DIP_assignment\\train\\images\\{temp}.bmp"
    mask_path= f"E:\\2. DIP\\2. LAB\\pythonProject\\Assignment\\dataset_DIP_assignment\\train\\masks\\{temp}.png"
    img=image_read(image_path)

    mask_img=image_read(mask_path)
    #show_image(img,1,"Original Image")
    frame_size=7
    padd_size=frame_size//2
    img1=frame_image(img,frame_size)
    #show_image(img1,1,"frame Image")
    hist = cal_histogram(img1)
    valid_pixels = img[img < 245]  # Ignore white background
    mean = np.mean(valid_pixels)
    std = np.std(valid_pixels)
    print("mean: ",mean)
    print("std: ",std)

    pdf_values = pdf(img1, hist)
    cdf = cummulative_pdf(pdf_values)
    transform_func = transform(cdf)
    nucl_thres=.5 * std
    cyto_thres = mean + (0.3* std)

    nuc_img,eq_list=connectivit_8(img1,nucl_thres,padd_size)
    cyto_img,eq_list1=connectivit_8(img1,cyto_thres,padd_size)
    nuc_img=overlap_label(nuc_img,cyto_img)
    """"
    plot_graph(hist, "Histogram", "Gray Shades", "Freq", "blue")
    plot_graph(pdf_values, "PDF", "Gray Shades", "Prob", "green")
    plot_graph(cdf, "CDF", "Gray Shades", "Cumulative Prob", "red")
    plot_graph(transform_func, "Trans Func", "Gray Shades", "y", "yellow")
    """
    print("nucl_thres: ", nucl_thres)
    print("cyto_thres: ", cyto_thres)

    nuc_mask=limit_img(nuc_img,255)
    cyto_mask=limit_img(cyto_img,128)
    #show_image(nuc_mask,1,'Nuc mask')
    #show_image(cyto_mask, 1, 'Cyto mask')
    mask=merge_mask(nuc_img,cyto_img)
    #show_image(mask,1,'MAde Mask')
    print ("dice coefficient of nucleus: ",dice_coef(mask_img,mask,128))
    print ("dice coefficient of cytoplasm: ",dice_coef(mask_img,mask,255))

i:  37
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 3579, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_18444\4050407075.py", line 190, in <module>
    img1=frame_image(img,frame_size)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dell\AppData\Local\Temp\ipykernel_18444\4050407075.py", line 43, in frame_image
    output_img[i - padd_size, j - padd_size] = normal_pixel(new_value,pix_min,pix_max)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OverflowError: Python integer -3 out of bounds for uint8

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 2170, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^